In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [1]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'edpb_guidelines-art_6-1-b-adopted_after_public_consultation_en.pdf'
paper_name = "Guidelines 2/2019 on the processing of personal data under Article 6(1)(b) GDPR in the context of the provision of online services to data subjects"
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

output_file = "../tmp/online_services.md"

In [4]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 3, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../../original/online_services.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [3]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'^(\d+) (?!\.)(.+)$')
subsection_pattern = re.compile(r'(\d\.\d)\s*(.*)') 
# subpoint_pattern = re.compile(r'^(\d+)\.\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^ANNEX (\d+) - (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            heading = True
            text = match.group(2)
            section_reference = subsection
            data.append([section, subsection, point, heading, text, section_reference])
        elif annex_match:
            match = annex_match
            annex_started = True
            section = "Annex"
            subsection = ""
            point = ""
            heading = True
            text = match.group(1)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)

#df.loc[df["section_reference"] == "",  "section_reference"] = "INTRODUCTION"
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [4]:
df[80:110]
#df.iloc[192]["text"]
#df[df["text"] == " "]

,section,subsection,point,heading,text,section_reference
80,,,,False,Example 6,2.7
81,,,,False,"In some cases, financial institutions have a d...",2.7
82,3,,,True,PART 3 - APPLICABILITY OF ARTICLE 6(1)(B) IN S...,3
83,3,3.1,,True,Processing for 'service improvement'[^25],3.1
84,,,,False,48. Online services often collect detailed inf...,3.1
85,,,,False,49. The EDPB does not consider that Article 6(...,3.1
86,3,3.2,,True,Processing for 'fraud prevention',3.2
87,,,,False,"50. As WP29 has previously noted,[^26] process...",3.2
88,3,3.3,,True,Processing for online behavioural advertising,3.3
89,,,,False,"51. Online behavioural advertising, and associ...",3.3


In [5]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 5 augmented with footnotes
Row 6 augmented with footnotes
Row 7 augmented with footnotes
Row 8 augmented with footnotes
Row 9 augmented with footnotes
Row 10 augmented with footnotes
Row 19 augmented with footnotes
Row 20 augmented with footnotes
Row 24 augmented with footnotes
Row 27 augmented with footnotes
Row 28 augmented with footnotes
Row 31 augmented with footnotes
Row 35 augmented with footnotes
Row 37 augmented with footnotes
Row 42 augmented with footnotes
Row 44 augmented with footnotes
Row 53 augmented with footnotes
Row 61 augmented with footnotes
Row 66 augmented with footnotes
Row 83 augmented with footnotes
Row 87 augmented with footnotes
Row 90 augmented with footnotes
Row 93 augmented with footnotes
Row 94 augmented with footnotes
Row 95 augmented with footnotes
Row 96 augmented with footnotes


In [6]:
file = "../../inputs/documents/online_services.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [9]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.online_services
importlib.reload(gdpr_rag.documents.online_services)
from gdpr_rag.documents.online_services import OnlineServices

path_to_manual_as_csv_file = "../../inputs/documents/online_services.parquet"

doc = OnlineServices(path_to_manual_as_csv_file)


In [10]:
from IPython.display import Markdown, display

section = "1"

section = "3.4"


print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


3 PART 3 - APPLICABILITY OF ARTICLE 6(1)(B) IN SPECIFIC SITUATIONS
3.4 Processing for personalisation of content[^34]


# 3 PART 3 - APPLICABILITY OF ARTICLE 6(1)(B) IN SPECIFIC SITUATIONS

## 3.4 Processing for personalisation of content[^34]

57. The EDPB acknowledges that personalisation of content may (but does not always) constitute an intrinsic and expected element of certain online services, and therefore may be regarded as necessary for the performance of the contract with the service user in some cases. Whether such processing can be regarded as an intrinsic aspect of an online service, will depend on the nature of the service provided, the expectations of the average data subject in light not only of the terms of service but also the way the service is promoted to users, and whether the service can be provided without personalisation. Where personalisation of content is not objectively necessary for the purpose of the underlying contract, for example where personalised content delivery is intended to increase user engagement with a service but is not an integral part of using the service, data controllers should consider an alternative lawful basis where applicable.

Example 7

An online hotel search engine monitors past bookings of users in order to create a profile of their typical expenditure. This profile is subsequently used to recommend particular hotels to the user when returning search results. In this case, profiling of user's past behaviour and financial data would not be objectively necessary for the performance of a contract, i.e. the provision of hospitality services based on particular search criteria provided by the user. Therefore, Article 6(1)(b) would not be applicable to this processing activity.

Example 8

An online marketplace allows potential buyers to browse for and purchase products. The marketplace wishes to display personalised product suggestions based on which listings the potential buyers have previously viewed on the platform in order to increase interactivity. This personalisation it is not objectively necessary to provide the marketplace service. Thus, such processing of personal data cannot rely on Article 6(1)(b) as a legal basis.

  
[^34]: Online services may also need to take into account Directive (EU) 2019/770 of the European Parliament and of the Council of 20 May 2019 on certain aspects concerning contracts for the supply of digital content and digital services (OJ L 136, 22.05.2019, p. 1), which will apply as from 1 January 2022.